In [1]:
import sqlite3 as sql
import pandas as pd

In [2]:
def rq(q):
    conn = sql.connect("stores.db")
    return pd.read_sql(q,conn)

In [3]:
q = """select name from sqlite_master
where type = 'table'

"""

table_names = rq(q)

In [5]:
table_names

,name
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


In [6]:
q = """select * from productlines

"""

productlines = rq(q)

In [7]:
productlines

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [92]:
# Low stock


q = """ select p.productCode,cast(sum(od.quantityOrdered)/p.quantityInStock as float) as "low_stock"
from products p
inner join orderdetails od on p.productCode=od.productCode
group by 1
order by 2 desc
limit 10


 
"""

rq(q)

,productCode,low_stock
0,S24_2000,67.0
1,S12_1099,13.0
2,S32_4289,7.0
3,S32_1374,5.0
4,S72_3212,2.0
5,S700_3167,1.0
6,S700_1938,1.0
7,S50_4713,1.0
8,S32_3522,1.0
9,S18_2795,1.0


In [91]:
# Product performance

q = """select p.productCode,
sum(quantityOrdered*priceEach) as "prod_per" from products p
inner join orderdetails od on 
p.productCode=od.productCode

group by 1
order by 2 desc
limit 10

"""

rq(q)

,productCode,prod_per
0,S18_3232,276839.98
1,S12_1108,190755.86
2,S10_1949,190017.96
3,S10_4698,170686.00
4,S12_1099,161531.48
5,S12_3891,152543.02
6,S18_1662,144959.91
7,S18_2238,142530.63
8,S18_1749,140535.60
9,S12_2823,135767.03


In [98]:
# combined query

q = """with firsttable as(
select p.productCode,cast(sum(od.quantityOrdered)/p.quantityInStock as float) as "low_stock"
from products p 
inner join orderdetails od on p.productCode=od.productCode

group by 1
order by 2 desc




),
secondtable as(select p.productCode,sum(quantityOrdered*priceEach) as "prod_per" from products p
inner join orderdetails od on 
p.productCode=od.productCode
group by 1
order by 2 desc




)
select ft.productCode,st.prod_per from firsttable ft
inner join secondtable st on ft.productCode=st.productCode
where ft.low_stock>0

order by st.prod_per desc 
limit 11



"""

rq(q)

,productCode,prod_per
0,S12_1099,161531.48
1,S18_2795,132275.98
2,S32_1374,89364.89
3,S700_3167,76618.40
4,S50_4713,73670.64
5,S700_1938,69531.61
6,S24_2000,67193.49
7,S32_4289,60493.33
8,S32_3522,57282.49
9,S72_3212,47550.40


In [108]:
# profit by customer
q = """select customerNumber,sum(-(buyprice*quantityOrdered-quantityOrdered*priceEach)) as "profit" from orders o
inner join orderdetails od
on od.orderNumber=o.orderNumber
inner join products p
on p.productCode=od.productCode
group by 1

 """

rq(q)

,customerNumber,profit
0,103,10063.80
1,112,31312.72
2,114,70311.07
3,119,60875.30
4,121,41391.52
...,...,...
93,486,33598.57
94,487,17230.12
95,489,10868.04
96,495,25244.69


In [164]:
# Top 5 VIP customers

q = """
select c.customerNumber,c.contactLastName,c.contactFirstName,c.city,c.country,sum(-(p.buyprice*od.quantityOrdered-od.quantityOrdered*od.priceEach)) as "profit" from customers c
inner join orders o
on c.customerNumber=o.customerNumber
inner join orderdetails od
on od.orderNumber=o.orderNumber
inner join products p
on p.productCode=od.productCode



group by 1
order by 6 desc
limit 5



 
 """
rq(q)

,customerNumber,contactLastName,contactFirstName,city,country,profit
0,141,Freyre,Diego,Madrid,Spain,326519.66
1,124,Nelson,Susan,San Rafael,USA,236769.39
2,151,Young,Jeff,NYC,USA,72370.09
3,114,Ferguson,Peter,Melbourne,Australia,70311.07
4,119,Labrune,Janine,Nantes,France,60875.30


In [166]:
# Top 5 less engaging customers


q = """select c.customerNumber,c.contactLastName,c.contactFirstName,c.city,c.country,sum(-(p.buyprice*od.quantityOrdered-od.quantityOrdered*od.priceEach)) as "profit" from customers c
inner join orders o
on c.customerNumber=o.customerNumber
inner join orderdetails od
on od.orderNumber=o.orderNumber
inner join products p
on p.productCode=od.productCode



group by 1
order by 6 asc
limit 5
 """
rq(q)

,customerNumber,contactLastName,contactFirstName,city,country,profit
0,219,Young,Mary,Glendale,USA,2610.87
1,198,Taylor,Leslie,Brickhaven,USA,6586.02
2,473,Ricotti,Franco,Milan,Italy,9532.93
3,103,Schmitt,Carine,Nantes,France,10063.80
4,489,Smith,Thomas,London,UK,10868.04


In [173]:
# Customer LTV
q = """

select sum(-(buyprice*quantityOrdered-quantityOrdered*priceEach))/count(distinct customerNumber) as "ltv" from orders o
inner join orderdetails od
on od.orderNumber=o.orderNumber
inner join products p
on p.productCode=od.productCode


 """

rq(q)

,ltv
0,39039.594388
